# Convolutional Neural Network and Adversarial Attacks

En el siguiente código se crea una red neuronal convolucional que identifica dígitos escritos a mano para posteriormente engañarla usando un white box adversarial attack conocido como FGSM

# Importaciones y definición de CPU o GPU

In [ ]:
import os
# os.environ["CUDA_VISIBLE_DEVICES"] = "1" # Poner -1 para CPU y comentar para GPU

import tensorflow as tf
from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt
import numpy as np
from random import randrange

print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU'))) # Imprimimos qué usará como comprobación

# Dataset

A continuación descargamos el dataset

In [ ]:
#  Cargamos el dataset y lo dividimos en entrenamiento y entreno
(train_images, train_labels), (test_images, test_labels) = datasets.mnist.load_data()

# Reducimos el dataset para hacer que la compilación sea más rápida
NUM_TRAINSET = 60000    # El número máximo es 60000
NUM_TESTSET = 10000       # El número máximo es 10000

train_images, train_labels = train_images[0:NUM_TRAINSET, :, :], train_labels[0:NUM_TRAINSET]
test_images, test_labels = test_images[0:NUM_TESTSET, :, :], test_labels[0:NUM_TESTSET]

# Importante: normalizar los píxeles entre 0 y 1. También se puede usar el método normalize
train_images, test_images = train_images / 255.0, test_images / 255.0

## Comprobamos el formato

Elegimos una imagen y mostramos su imagen y su etiqueta

In [ ]:
num_imagen = 7  

plt.imshow(train_images[num_imagen] ,cmap=plt.cm.binary)
plt.xlabel(train_labels[num_imagen])
plt.show()

## Reshape
La red espera una entrada de la forma (60000, 28, 28, 1), de forma que tenemos que modificar la entrada a eso

In [ ]:
train_images = train_images.reshape(NUM_TRAINSET,28,28,1)
test_images = test_images.reshape(NUM_TESTSET,28,28,1)

# Definición de la red

In [ ]:
# Creamos un modelo secuencial, es decir, la información fluirá de izquierda a derecha
model = models.Sequential()

# Añadimos las capas convolucionaes
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))

# Y posteriormente, las pasamos a capas lineales
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu')) # Que escojamos 64 neuronas no tiene nada que ver con lo anterior
model.add(layers.Dense(10, activation='softmax'))

# Echemos un vistazo a cómo nos queda la red
model.summary()

## Creación de la red

In [ ]:
model.compile(optimizer='adam', loss=tf.keras.losses.SparseCategoricalCrossentropy(), metrics=['accuracy'])

## Entrenamiento de la red

In [ ]:
entrenamiento = model.fit(train_images, train_labels, batch_size = 32, epochs=3, validation_data=(test_images, test_labels))

## Análisis de resultados

Comprobamos ahora en función de los datos de pérdida y precisión si sería beneficial aumentar las épocas o no

In [ ]:
# Mostramos la pérdida y la precisión
plt.figure(figsize=(10,4))

plt.subplot(121)
plt.title("Loss")
plt.xlabel("Época")
plt.plot(entrenamiento.history['loss'], label= "Loss")

plt.subplot(122)
plt.title("Precisión")
plt.xlabel("Época")
plt.plot(entrenamiento.history['accuracy'], label= "Precisión")

En base a la gráfica se observa que el valor ideal de épocas es 3 (empezamos en 0)

### Guardar datos en txt

Guardamos los resultados en un txt para poder crear las figuras en latex

In [ ]:
# Creamos un vector x con el número de epocas desde 1
x = np.linspace(1, len(entrenamiento.history['loss']), num =len(entrenamiento.history['loss']))
        
# Formamos una matriz y la transponemos para que los vectores se queden como columnas
matriz = np.array([x, entrenamiento.history['loss']])
matriz = matriz.transpose()
        
# Cuardamos el txt definiendo las columnas, el formato, el delimitador y evitando que aparezca el hastagh
np.savetxt("mnistbasic_loss.txt", matriz, header = 'Epoch Loss', fmt='%1.10f', delimiter = ' ', comments='')

# Repetimos para la precisión
matriz = np.array([x, entrenamiento.history['accuracy']])
matriz = matriz.transpose()
np.savetxt("mnist_basic_accuracy.txt", matriz, header = 'Epoch Accuracy', fmt='%1.10f', delimiter = ' ', comments='')

# Predicción de resultados

Comprobamos que las imágenes introducidas corresponden con los valores predichos


In [ ]:
num_imagen = 0

plt.imshow(test_images[num_imagen], cmap=plt.cm.binary)
plt.xlabel(test_labels[num_imagen])
plt.show()

print("La clase predicha es:", np.argmax(model.predict([test_images])[num_imagen]), "con una confianza del", 
      round(np.max(model.predict([test_images])[num_imagen])*100, 2), "%")

# Creación del dataset FGSM
### Crear las perturbaciones
Primero calculamos las perturbaciones para el dataset escogido, de forma que podemos realizar el FGSM sobre el test para realizar pruebas o sobre el train para entrenar posteriormente a la red con él

In [ ]:
# Introducimos el dataset del que queremos imágenes adversariales. De momento es test, luego ya en la defensa pondremos train
dataset = "test"
   
perturbaciones_testset = FGSM(dataset)

### Creación de las adversarial images
Una vez tenemos las perturbaciones del dataset escogido, lo recorremos para los valores de epsilon deseados obteniendo un vector de predicciones

In [ ]:
# Definimos los valores de epsilon para los que queremos crear el dataset
epsilons = np.array([0.01, 0.05, 0.1, 0.15, 0.2, 0.3, 0.4, 0.5])

# Creamos nuestro vector vacío con las predicciones
predicciones = np.empty([len(epsilons), NUM_TESTSET, 10])

# Creamos nuestro vector vacío que contendrá el testset
adversarial_testset = np.empty([len(epsilons), NUM_TESTSET, 28, 28, 1])
            
# Recorremos los datasets para cada uno de esos valores de epsilon    
for i in range(len(epsilons)):
    # Creamos el adversarial dataset
    adversarial_testset[i] = creacion_adversarial_dataset(epsilons[i], dataset, perturbaciones_testset)   

    # Calculamos las métricas
    predicciones[i] = model.predict([adversarial_testset[i]])

### Comprobación de que se han creado correctamente

In [ ]:
# Mostramos la forma
print("La forma del vector predicciones es:", predicciones.shape)
print("La forma del adversarial testset es:", adversarial_testset.shape)
    
# Imprimimos una imagen aleatoria con un valor de epsilon aleatorio
num_imagen = randrange(NUM_TESTSET)
valor = randrange(len(epsilons))
    
imagen_original = test_images[num_imagen]
perturbacion = perturbaciones_testset[num_imagen] * 0.5 + 0.5
imagen = adversarial_testset[valor][num_imagen]
    
# Llamamos a la funciones de impresión y predicción
impresion_imagen_adversarial(imagen_original, perturbacion, imagen, epsilons[valor])
prediccion_adversarial(imagen)

# Análisis de métricas

In [ ]:
# Calculamos los resultados. IMPORTANTE: Poner 1 si estamos trabajando con el dataset adversativo ó 0 si no, ya que creará los txt
# con el nombre correspondiente
metricas(predicciones, NUM_TESTSET, adversativo = 0)

# Defensa frente a ataques adversativos
A continuación tenemos que realizar un proceso muy parecido al anterior pero ahora con el trainset, de forma que obtendremos 60000 imágenes adversativas para el valor o valores de epsilon especificados. Esas imágenes las añadiremos al dataset inicial y entrenaremos a nuestra red con los conjuntos originales y los adversariales de forma que la precisión debería volver a subir

### Creación de las perturbaciones del trainset

In [ ]:
dataset = "train"
   
perturbaciones_trainset = FGSM(dataset)

### Creación del trainset adversarial

In [ ]:
# Definimos los valores de epsilon para los que queremos crear el dataset
# Tarda un huevo, así que, de momento, preferiblemente sólo 1. Pongo 0.4 porque es el que hace que sea básicamente random
epsilon = np.array([0.4])

# Creamos nuestro vector vacío que contrendrá el dataset
adversarial_trainset = np.empty([len(epsilon), NUM_TRAINSET, 28, 28, 1])
            
# Recorremos los datasets para cada uno de esos valores de epsilon    
for i in range(len(epsilon)):
    # Creamos el adversarial dataset
    adversarial_trainset[i] = creacion_adversarial_dataset(epsilon[i], dataset, perturbaciones_trainset)   

### Comprobación de que se ha creado correctamente

In [ ]:
# Mostramos la forma
print("La forma del adversarial trainset es:", adversarial_trainset.shape)
    
# Imprimimos una imagen aleatoria con un valor de epsilon aleatorio
valor = randrange(len(epsilon))
num_imagen = randrange(NUM_TRAINSET)

imagen_original = train_images[num_imagen]
perturbacion = perturbaciones_trainset[num_imagen]
imagen = adversarial_trainset[valor][num_imagen]

# Llamamos a la funciones de impresión y predicción
impresion_imagen_adversarial(imagen_original, perturbacion, imagen, epsilon[valor])

### Creación del nuevo conjunto de entrenamiento y test con las imágenes adversariales
Hasta ahora tenemos nuestro adversarial trainset y el adversarial dataset. Ahora tenemos que definir qué imágenes queremos de dentro de esos conjuntos, así como establecer sus correctas etiquetas para entrenar posteriormente la red

In [ ]:
# Creación del conjunto de entrenamiento
# Lo inicializamos con las imágenes originales
trainset_defensa = train_images
trainset_defensa_labels = train_labels

# Y vamos añadiendo los epsilons establecidos, aunque típicamente será sólo 1
for i in range(len(epsilon)):
    trainset_defensa = np.concatenate((trainset_defensa, adversarial_trainset[i]))
    trainset_defensa_labels = np.concatenate((trainset_defensa_labels, train_labels))

# Imprimimos las formas para comprobar que son las correctas
print(trainset_defensa.shape)
print(trainset_defensa_labels.shape)

In [ ]:
# Creación del conjunto de test. Lo creamos para los mismos epsilons
testset_defensa = test_images
testset_defensa_labels = test_labels

# Encontramos la posición de epsilon en el vector de epsilons
posicion_epsilon = int(np.where(epsilons == epsilon)[0])

# Y vamos añadiendo el testset que corresponde al mismo valor de epsilon que en el caso anterior, que es la posición 6
testset_defensa = np.concatenate((testset_defensa, adversarial_testset[posicion_epsilon]))
testset_defensa_labels = np.concatenate((testset_defensa_labels, test_labels))

# Imprimimos las formas para comprobar que son las correctas
print(testset_defensa.shape)
print(testset_defensa_labels.shape)

### Entrenamiento del modelo con el nuevo dataset adversarial

In [ ]:
# Antes de entrenar al modelo, volvemos a compilarlo para asegurarnos de que se reinicia
model.compile(optimizer='adam', loss=tf.keras.losses.SparseCategoricalCrossentropy(), metrics=['accuracy'])

# Y entrenamos
entrenamiento = model.fit(trainset_defensa, trainset_defensa_labels, epochs=3, validation_data=(testset_defensa, testset_defensa_labels), batch_size = 32)

### Cálculo de las nuevas predicciones del modelo

In [ ]:
# Creamos nuestro vector vacío con las predicciones
predicciones = np.empty([len(epsilons), NUM_TESTSET, 10])
            
# Recorremos los datasets para cada uno de esos valores de epsilon    
for i in range(len(epsilons)):

    # Calculamos las métricas
    predicciones[i] = model.predict([adversarial_testset[i]])

### Análisis de métricas con el nuevo modelo
Repetimos los pasos anteriores para comprobar si efectivamente hemos mejorado la precisión

In [ ]:
# Volvemos a analizar el modelo pero especificamos que estamos con el dataset adversaativo para que cree los txt
metricas(predicciones, NUM_TESTSET, adversativo = 1)

# Funciones utilizadas

###  Creación de la perturbación

In [ ]:
def FGSM(dataset):
    
    # Definimos el dataset a utilizar, si el train o el test
    if dataset == "test":
        dataset = test_images
        labels = test_labels
    else:
        dataset = train_images
        labels = train_labels
    
    # Creamos un vector vacío para almacenar las perturbaciones
    perturbaciones = np.empty([len(dataset), 28, 28, 1])
    
    # Recorremos las imágenes del dataset elegido
    for num_imagen in range(len(dataset)):
        # Expandimos la dimensión
        imagen = dataset[num_imagen]
        imagen = np.expand_dims(imagen, axis = 0)
        label = labels[num_imagen]

        # Convertimos a tensor
        imagen = tf.convert_to_tensor(imagen, dtype = tf.float32)

        # Definimos la función de pérdida
        funcion_loss = tf.keras.losses.SparseCategoricalCrossentropy()

        # Creamos la perturbacion
        def crear_perturbacion(imagen, label):
            with tf.GradientTape() as tape:
                tape.watch(imagen)
                prediccion = model(imagen)
                loss = funcion_loss(label, prediccion)

            gradiente = tape.gradient(loss, imagen)
            signo_grad = tf.sign(gradiente)
            return signo_grad

        # Llamamos a la función
        perturbaciones[num_imagen] = crear_perturbacion(imagen, label)

    return perturbaciones

### Creación del dataset adversarial

In [ ]:
def creacion_adversarial_dataset(epsilon, dataset, perturbaciones):
    
     # Definimos el dataset a utilizar, si el train o el test
    if dataset == "test":
        dataset = test_images
        labels = test_labels
    else:
        dataset = train_images
        labels = train_labels
    
    # Creamos el adversarial dataset vacío
    adversarial_dataset = np.empty([len(dataset), 28, 28, 1])
    
    # Recorremos el dataset original y le vamos sumando el vector de perturbaciones creado anteriormente
    for num_imagen in range(len(dataset)):
        adversarial_dataset[num_imagen] = dataset[num_imagen] + epsilon * (perturbaciones[num_imagen] * 0.5 + 0.5)
        adversarial_dataset[num_imagen] = tf.clip_by_value(adversarial_dataset[num_imagen], -1, 1)
    
    return adversarial_dataset

### Impresión del dataset adversarial para cada epsilon

In [ ]:
def impresion_imagen_adversarial(imagen_original, perturbacion, imagen, epsilon):
        
    # Imprimimos una imagen con su perturbación y su imagen adversarial resultante
    plt.figure(figsize=(15,4))

    plt.subplot(131)
    plt.title("Imagen original")
    plt.imshow(imagen_original, cmap=plt.cm.binary)

    plt.subplot(132)
    plt.title("Perturbación")
    plt.imshow(perturbacion, cmap=plt.cm.binary)

    plt.subplot(133)
    plt.title("Imagen adversarial epsilon = {}".format(epsilon))
    plt.imshow(imagen, cmap=plt.cm.binary)

### Predicción de una imagen adversarial

In [ ]:
def prediccion_adversarial(imagen):
    # Primero expandimos una dimensión
    imagen = np.expand_dims(imagen, axis = 0)
    
    # Y realizamos la predicción
    print("La predicción es:", np.argmax(model.predict(imagen)), "con una confianza del", 
          round(np.max(model.predict(imagen)),2)*100, "%")

### Análisis de métricas

In [ ]:
def metricas(predicciones, numero_imagenes, adversativo):

    # Creamos los vectores globales
    aciertos_epsilon = np.empty(len(epsilons)) 
    fallos_epsilon = np.empty(len(epsilons))
    precisiones = np.empty(len(epsilons))
    confianzas = np.empty(len(epsilons)) 

    # Recorremos los epsilons
    for i in range(len(epsilons)):
        aciertos = 0
        confianza = 0

        # Recorremos las predicciones
        for j in range(numero_imagenes):
            prediccion = np.argmax(predicciones[i][j])
            confianza = confianza + np.max(predicciones[i][j])
            if prediccion == test_labels[j]:
                aciertos += 1

        # Calculamos las métricas para cada epsilon
        fallos = numero_imagenes - aciertos
        precision = (aciertos / numero_imagenes) * 100
        confianza = (confianza / numero_imagenes) * 100
        print("Para epsilon = {}: Aciertos = {}, Fallos = {} luego Precisión = {:0.2f} % con una confianza media del {:0.2f} %"
              . format(epsilons[i], aciertos, fallos, precision, confianza))

        # Creamos un vector con todas las métricas para poder realizar gráficas
        aciertos_epsilon[i] = aciertos
        fallos_epsilon[i] = fallos
        precisiones[i] = precision
        confianzas[i] = confianza

    # Creamos gráficas de los resultados
    plt.figure(figsize=(20,4))

    plt.subplot(141)
    plt.title("Aciertos")
    plt.xlabel("Epsilon")
    plt.ylabel("Nº aciertos")
    plt.plot(epsilons, aciertos_epsilon)

    plt.subplot(142)
    plt.title("Fallos")
    plt.xlabel("Epsilon")
    plt.ylabel("Nº fallos")
    plt.plot(epsilons, fallos_epsilon)

    plt.subplot(143)
    plt.title("Precisión")
    plt.xlabel("Epsilon")
    plt.ylabel("Porcentaje")
    plt.plot(epsilons, precisiones)

    plt.subplot(144)
    plt.title("Confianza")
    plt.xlabel("Epsilon")
    plt.ylabel("Porcentaje")
    plt.plot(epsilons, confianzas)
    
    # Guardamos todos los datos en un txt
    if adversativo == 0:
        nombre = "mnist_test_"
    else:
        nombre = "mnist_adversarial_"
    
    # Formamos una matriz y la transponemos para que los vectores se queden como columnas
    matriz = np.array([epsilons, aciertos_epsilon])
    matriz = matriz.transpose()
    np.savetxt(nombre + "aciertos.txt", matriz, header = 'Epsilon Aciertos', fmt='%1.10f', delimiter = ' ', comments='')
                       
    matriz = np.array([epsilons, fallos_epsilon])
    matriz = matriz.transpose()
    np.savetxt(nombre + "fallos.txt", matriz, header = 'Epsilon Fallos', fmt='%1.10f', delimiter = ' ', comments='')
                                          
    matriz = np.array([epsilons, precisiones])
    matriz = matriz.transpose()
    np.savetxt(nombre + "accuracy.txt", matriz, header = 'Epsilon Accuracy', fmt='%1.10f', delimiter = ' ', comments='')
                                                             
    matriz = np.array([epsilons, confianzas])
    matriz = matriz.transpose()
    np.savetxt(nombre + "confianza.txt", matriz, header = 'Epsilon Confianza', fmt='%1.10f', delimiter = ' ', comments='')
